In [ ]:
import os
import time
import cv2
import torch
from PIL import Image
from torchvision.models.detection import fasterrcnn_resnet50_fpn, FasterRCNN_ResNet50_FPN_Weights
from torchvision.transforms import functional as F

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

In [ ]:

weights = FasterRCNN_ResNet50_FPN_Weights.DEFAULT
model = fasterrcnn_resnet50_fpn(weights=weights)
model.eval()
# COCO class 17 = cat
CAT_CLASS_ID = 17

In [ ]:
def resize_with_padding(image, size=(224, 224)):
    return ImageOps.pad(image, size, color=(0, 0, 0))  

def crop(image_path):
    
    try:
        image = Image.open(image_path).convert("RGB")
    except Exception as e:
        print(f"Didnt load {image_path}: {e}")
        return None

    image_tensor = F.to_tensor(image)

    #  detection
    with torch.no_grad():
        outputs = model([image_tensor])[0]

    boxes = outputs['boxes']
    labels = outputs['labels']
    scores = outputs['scores']

    # Load image with OpenCV for cropping
    img_cv = cv2.imread(image_path)
    if img_cv is None:
        print(f"didnt load picture with cv2: {image_path}")
        return None
    h, w = img_cv.shape[:2]

    os.makedirs("cropped", exist_ok=True)

    found = False
    for i, (box, label, score) in enumerate(zip(boxes, labels, scores)):
        if label.item() == CAT_CLASS_ID and score.item() >= 0.8: #confidence level at more than 80%
            found = True
            x1, y1, x2, y2 = map(int, box.tolist())
            x1, y1, x2, y2 = max(0, x1), max(0, y1), min(w, x2), min(h, y2)
            cat_crop_cv = img_cv[y1:y2, x1:x2]

            cat_crop = Image.fromarray(cv2.cvtColor(cat_crop_cv, cv2.COLOR_BGR2RGB))
            # resized_crop = resize_with_padding(cat_crop_pil, size=(224, 224))

    if not found:
        print(f"no cats in: {image_path}")
        return None

    return cat_crop

In [ ]:

def screenshot_tiktok_hashtag(hashtag, num_posts, screenshots_per_post):
    service = Service(ChromeDriverManager().install())
    options = webdriver.ChromeOptions()
    user_data_dir = r"C:\SeleniumChromeProfile"
    options.add_argument(f"--user-data-dir={user_data_dir}")
    driver = webdriver.Chrome(service=service, options=options)

    time.sleep(1)
    driver.get(f"https://www.tiktok.com/tag/{hashtag}")
    time.sleep(5)

    # main folder for hashtag
    if not os.path.exists(hashtag):
        os.mkdir(hashtag)

    try:
        video_thumb = WebDriverWait(driver, 15).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "div.css-41hm0z"))
        )
        driver.execute_script("arguments[0].click();", video_thumb)
        print("clicked")
    except Exception as e:
        print("didnt click ", e)

    #subfolders
    for i in range(num_posts):
        video_folder = os.path.join(hashtag, str(i))
        os.makedirs(video_folder, exist_ok=True)

        for j in range(screenshots_per_post):
            screenshot_path = os.path.join(video_folder, f'shot{j+1}.png')
            driver.save_screenshot(screenshot_path)
            print(f'saved screenshot: {screenshot_path}')
            time.sleep(0.5)

        try:
            next_button = driver.find_element(By.CSS_SELECTOR, 'button[data-e2e="arrow-right"]')
            next_button.click()
            print(f'next video ({i+2})')
        except Exception as e:
            print(f"did not go to next video: {e}")
            break

        time.sleep(3)

    driver.quit()

In [ ]:
screenshot_tiktok_hashtag('catcaredit', num_posts=55, screenshots_per_post=30)


In [ ]:


# Input and output folders
input_base = r'C:\Users\Marta\Documents\projekty\cat app\catcaredit'
output_base = r'C:\Users\Marta\Documents\projekty\cat app\cropped'

# Traverse each subfolder
for folder_name in os.listdir(input_base):
    input_subfolder = os.path.join(input_base, folder_name)
    output_subfolder = os.path.join(output_base, folder_name)

    if os.path.isdir(input_subfolder):
        os.makedirs(output_subfolder, exist_ok=True)

        for filename in os.listdir(input_subfolder):
            if filename.lower().endswith((".png", ".jpg", ".jpeg")):
                input_path = os.path.join(input_subfolder, filename)
                output_path = os.path.join(output_subfolder, filename)

                cropped_img = crop(input_path)
                if cropped_img is not None:
                    cropped_img.save(output_path)
                    print(f'saved cropped image: {output_path}')



